In [1]:
import scipy.io
import numpy
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [2]:
data = scipy.io.loadmat("AC_DataType3_LorR1_numVoxels235_nOrients9_AC-LH-V1_Betas.mat")

In [3]:
print (data.keys())

dict_keys(['o', 'g', '__version__', 'scans', '__header__', '__globals__', 'bEst'])


In [4]:
bEst = data['bEst']
scans = data['scans']
orientations = data['o']
conditions = data['g']

print (bEst)

[[ 0.14734871 -0.40623407 -0.11559177 ...  1.65306927  0.80184849
   0.13153074]
 [-0.25418487  1.48078863  1.28284947 ...  1.66282417  0.49568171
   0.44276535]
 [ 0.32321879  0.14927317  0.10689787 ...  0.66414063 -0.21947812
   0.33840493]
 ...
 [-0.43396752 -0.5527465  -0.35209847 ...  0.74607369  0.55405472
  -1.22763765]
 [ 0.42516445 -0.76820557 -0.09765152 ...  1.07472235  0.33174269
  -0.71472298]
 [-0.96992205  0.78093756  0.03702796 ...  1.27003607  0.38358051
   0.65319594]]


In [53]:
beta_tensor = torch.from_numpy(bEst)
beta_tensor.requires_grad = True
orientations_tensor = torch.from_numpy(orientations[:, 0])

In [54]:
target = (torch.from_numpy(orientations)-1).float()

target.requires_grad = False

In [55]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(235, 6)
        self.fc2 = nn.Linear(6, 9)
        
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        #print (x)
        x = torch.sigmoid(self.fc2(x))
        return x
    

In [56]:
model = Model()

In [88]:
out = model(beta_tensor.float())

In [89]:
criterian = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [90]:

loss = criterian(out, target[:, 0].long())
loss.backward(retain_graph=True)

In [91]:
optimizer.step()

In [92]:
print (loss)

tensor(2.2015, grad_fn=<NllLossBackward>)
